In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from data_cleansing import Wrangling_data
from custom_transformer import Streak_score_wavg, GD_weight_avg
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from database import SQLiteDBManager

# sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn import set_config
# listing model 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from supreme_cross_validation import HyperclassifierSearch

# pipline
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Normalize data 
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler

# imbalance data
from imblearn.over_sampling import  SMOTE, RandomOverSampler
from sklearn.model_selection import RepeatedStratifiedKFold

# Save model
import pickle
import joblib
import os
import time
from datetime import datetime

# Visual the confusion matrix
import matplotlib.pyplot as plt

In [3]:
import warnings

# Filter or ignore specific warning category
warnings.filterwarnings("ignore")

# Calculate

In [4]:
qa = 10**(2072/400)
qb =  10**(1866/400)
Ea = qa/(qa+qb)
Eb = qb/(qa+qb)

In [5]:
Ea

0.7659946707664886

In [6]:
Eb

0.2340053292335114

In [7]:
Ra_ = 2072 + 30*(-Ea)
Rb_ = 1866 + 30*(1-Eb)

In [8]:
Ra_

2049.0201598770054

In [9]:
Ra_

2049.0201598770054

In [10]:
Rb_

1888.9798401229946

In [11]:
1866 - 1860 

6

In [12]:
Rb_

1888.9798401229946

In [13]:
1890 - 1866

24

# Matches 

In [14]:
path = 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data'
data_list = [os.path.join(path,file) for file in os.listdir(path) if os.path.splitext(file)[-1] == '.csv']
data_list

["C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\2019 FIFA Women's World Cup_detail_match.csv",
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\2021 Friendlies (W)_detail_match.csv',
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\2022 Friendlies (W)_detail_match.csv',
 "C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\AFC Women's Asian Cup qualification_detail_match.csv",
 "C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\AFC Women's Asian Cup_detail_match.csv",
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\Africa Women Cup of Nations_detail_match.csv',
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\Algarve Cup_detail_match.csv',
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\CONCACAF W Championship_detail_match.csv',
 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data\\Copa América Femenina_detail_match.csv',
 "C:/Users/user2/PycharmProjects/selenium_scraping

## Combine data frame

In [15]:
dfs = [pd.read_csv(filepath_or_buffer=f,parse_dates=['Date']) for f in data_list]

In [16]:
full_df = pd.concat(dfs,ignore_index=True).sort_values(by='Date').reset_index(drop=True)
full_df

,Tournament,Round,Day,Date,Time,Home,Away,score_home,score_away,pen_home,pen_away,result,pen_result,Venue
0,2019 FIFA Women's World Cup,Group stage,Fri,2019-06-07 21:00:00,21:00,France,Korea Rep,4,0,0,0,H,0,Parc des Princes
1,2019 FIFA Women's World Cup,Group stage,Sat,2019-06-08 15:00:00,15:00,Germany,China PR,1,0,0,0,H,0,Roazhon Park
2,2019 FIFA Women's World Cup,Group stage,Sat,2019-06-08 18:00:00,18:00,Spain,South Africa,3,1,0,0,H,0,Stade Océane
3,2019 FIFA Women's World Cup,Group stage,Sat,2019-06-08 21:00:00,21:00,Norway,Nigeria,3,0,0,0,H,0,Stade Auguste-Delaune II
4,2019 FIFA Women's World Cup,Group stage,Sun,2019-06-09 13:00:00,13:00,Australia,Italy,1,2,0,0,A,0,Stade du Hainaut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,FIFA Women's World Cup,Quarter-finals,Fri,2023-08-11 19:30:00,19:30,Japan,Sweden,1,2,0,0,A,0,Eden Park (Neutral Site)
1452,FIFA Women's World Cup,Quarter-finals,Sat,2023-08-12 17:00:00,17:00,Australia,France,0,0,7,6,D,H,Suncorp Stadium (Neutral Site)
1453,FIFA Women's World Cup,Quarter-finals,Sat,2023-08-12 20:30:00,20:30,England,Colombia,2,1,0,0,H,0,Accor Stadium (Neutral Site)
1454,FIFA Women's World Cup,Semi-finals,Tue,2023-08-15 20:00:00,20:00,Spain,Sweden,2,1,0,0,H,0,Eden Park (Neutral Site)


# Create clean data for features engineering purpose

In [17]:
path = 'C:/Users/user2/PycharmProjects/selenium_scraping_match/data'
raw_df = Wrangling_data(data_path=path)
clean_df = raw_df.df_wrangling(update_league_tb=True, update_team_perf=True)

Connect database ./database/soccer_database.db successfully
Tournament is updated successfully
DataFrame is import into 'league_tb' in './database/soccer_database.db' successfully.

>>>>Update League table successfully.

 Table 'league_tb' is exported to Dataframe successfully.
 Table 'fifa_rank_tb' is exported to Dataframe successfully.
Team performance is updated successfully
DataFrame is import into 'team_performance' in './database/soccer_database.db' successfully.

>>>>Update Team Performance table successfully.

 Table 'team_performance' is exported to Dataframe successfully.


In [18]:
clean_df['Tournament_id'].value_counts(sort=False)

Tournament_id
5      52
6     204
7     206
13    118
9     242
2      26
3      25
8     232
1       6
11     28
14     16
15     31
12     25
10    177
4       6
16     62
Name: count, dtype: int64

# Build Feature Engineer Pipeline

## Define

In [19]:
def get_feature(df:pd.DataFrame, is_train:bool=False):
    df['Time'] = df['Date'].dt.time
    df['Date'] = pd.to_datetime(df['Date'].dt.date)
    features_col = ['Date', 'Time', 'Tournament_id', 'round_id', 'Home', 'Away', 'home_perf_wm', 'away_perf_wm',
                        'rank_diff', 'Home_streak_score', 'Away_streak_score', 'Home_streak_length',
                        'Away_streak_length', 'Home_diff_score', 'Away_diff_score']
    if is_train:
        features_col.extend(['result','pen_result'])
        filter_length = (df['Home_streak_length'] >= 5) & (df['Away_streak_length'] >= 5)
        df = df.loc[filter_length, features_col].reset_index(drop=True).drop(columns=['Home_streak_length','Away_streak_length'])
        return df
    else:
        df = df[features_col].drop(columns=['Home_streak_length','Away_streak_length'])
        return df

## feature engineer

In [20]:
streak_feat_list = ['Home_streak_score', 'Away_streak_score', 'Home_streak_length', 'Away_streak_length']
GD_feat_list = ['Home_diff_score', 'Away_diff_score']

In [21]:
feature_engineer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='mean', missing_values=np.nan)),
    ('merger', Merge_rank_league_perform()),
    ('streak_score',Streak_score_wavg(df=clean_df, feat_names=streak_feat_list,n=6)),
    ('goal_diff',GD_weight_avg(df=clean_df, feat_names=GD_feat_list,n=6))
])
feature_engineer.fit(full_df)


>>>>Merge_rank_league_perform.fit() called.


>>>>Merge_rank_league_perform.transform() called.

Connect database ./database/soccer_database.db successfully
 Table 'league_tb' is exported to Dataframe successfully.
 Table 'fifa_rank_tb' is exported to Dataframe successfully.
 Table 'team_performance' is exported to Dataframe successfully.

>>>>Closed DataBase successfully.


>>>>Finish Merge_rank_league_perform.transform().


>>>>Streak_score_wavg.fit() called.


>>>>Streak_score_wavg.transform() called.


>>>>Finish Streak_score_wavg.transform().


>>>>GD_weight_avg.fit() called.



Pipeline(steps=[('merger', Merge_rank_league_perform()),
                ('streak_score',
                 Streak_score_wavg(df=      tour_start  Tournament_id  round_id                Date       Home  \
0           2019              5         1 2019-06-07 21:00:00     France   
1           2019              5         1 2019-06-08 15:00:00    Germany   
2           2019              5         1 2019-06-08 18:00:00      Spain   
3           2019              5         1 2019-06-08 21:00:00     Norway   
4           2019              5         1 2019-06-09 13:00:00  Australia   
...          ...            ...       ...                 ...        ...   
1451        2023             16         3 2023-08-11 19:30:00      Japan   
1452        2023...
2        1471.52         1         0      0.923079      0.407767     530.76  
3        1554.94         1         0      0.737805      0.404348     353.31  
4        1846.50         0         1      0.401938      0.473333      73.19  
...          ...       ...       ...           ...           ...        ...  
1451     2049.71         0         1      0.705499      0.501471    -133.03  
1452     2026.65         0         0      0.401938      0.387168    -106.96  
1453     1702.64         1         0      0.716496      0.368394     338.12  
1454     2049.71         1         0      0.923079      0.501471     -47.43  
1455     2040.76         0         1      0.401938      0.914894    -121.07  

[1456 rows x 17 columns],
                               feat_names=['Home_diff_score',
                                           'Away_diff_score'],
                               n=6))])

## Fit with train data

In [22]:
full_df_pre = feature_engineer.transform(full_df)


>>>>Merge_rank_league_perform.transform() called.

Connect database ./database/soccer_database.db successfully
 Table 'league_tb' is exported to Dataframe successfully.
 Table 'fifa_rank_tb' is exported to Dataframe successfully.
 Table 'team_performance' is exported to Dataframe successfully.

>>>>Closed DataBase successfully.


>>>>Finish Merge_rank_league_perform.transform().


>>>>Streak_score_wavg.transform() called.


>>>>Finish Streak_score_wavg.transform().


>>>>GD_weight_avg.transform() called.


>>>>Finish GD_weight_avg.transform().



In [23]:
final_df = get_feature(df=full_df_pre, is_train=True)
final_df

,Date,Time,Tournament_id,round_id,Home,Away,home_perf_wm,away_perf_wm,rank_diff,Home_streak_score,Away_streak_score,Home_diff_score,Away_diff_score,result,pen_result
0,2019-07-02,21:00:00,5,4,England,USA,0.716496,0.555556,-49.27,1.000000,1.000000,1.990424,3.429089,A,0
1,2019-07-03,21:00:00,5,4,Netherlands,Sweden,0.834743,0.501471,-69.24,1.030842,0.949214,1.362714,0.959822,H,0
2,2019-07-06,17:00:00,5,5,England,Sweden,0.716496,0.501471,-8.95,0.936998,0.719511,1.415979,0.579900,A,0
3,2019-07-07,17:00:00,5,6,USA,Netherlands,0.910757,0.479614,109.56,1.000000,1.154977,2.974879,1.293187,H,0
4,2020-03-05,18:30:00,6,1,Malta,Georgia,0.420000,0.000000,131.97,-0.945487,-1.000000,-3.010254,-5.801186,H,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,2023-08-11,19:30:00,16,3,Japan,Sweden,0.705499,0.501471,-133.03,0.979878,0.395191,2.590036,1.032195,A,0
979,2023-08-12,17:00:00,16,3,Australia,France,0.401938,0.387168,-106.96,0.054874,0.572759,1.436822,1.393902,D,H
980,2023-08-12,20:30:00,16,3,England,Colombia,0.716496,0.368394,338.12,0.307864,0.523335,0.762022,0.778454,H,0
981,2023-08-15,20:00:00,16,4,Spain,Sweden,0.923079,0.501471,-47.43,0.784536,0.875788,1.393845,1.468319,H,0


# Define function to update ELO rank 

## Get list of national teams

In [58]:
database = SQLiteDBManager()

Connect database ./database/soccer_database.db successfully


In [25]:
database.list_tables_in_database()

Tables in the database:


['home_performance',
 'nation_tb',
 'fifa_rank_tb',
 'match_tb',
 'league_tb',
 'team_performance']

In [26]:
fifa_table = database.export_table_to_dataframe(table_name='fifa_rank_tb')
fifa_table

 Table 'fifa_rank_tb' is exported to Dataframe successfully.


,RK,team,team_short,total_point
0,1,USA,USA,2090.03
1,2,Germany,GER,2061.56
2,3,Sweden,SWE,2049.71
3,4,England,ENG,2040.76
4,5,France,FRA,2026.65
...,...,...,...,...
183,184,Madagascar,MAD,691.00
184,185,Anguilla,AIA,687.55
185,186,Turks and Caicos Islands,TCA,664.95
186,187,Sudan,SSD,649.69


In [27]:
team_dict = {i:1500 for i in fifa_table['team'].values}
list(team_dict.items())[:5]

[('USA', 1500),
 ('Germany', 1500),
 ('Sweden', 1500),
 ('England', 1500),
 ('France', 1500)]

## Define Function

In [57]:
def update_ELO_point(home, away,result:str,weight):
    global team_dict

    # Get team point before the match
    if home in team_dict.keys() and away in team_dict.keys():
        home_point = team_dict[home] 
        away_point = team_dict[away] 
    else:
        home_point =1500
        away_point = 1500

    # Chech result to get corresponding point
    if result == 'H':
        Aa = 1
        Ab = 0
    elif result == 'A':
        Aa = 0
        Ab = 1
    else:
        Aa = 0.5
        Ab = 0.5

    # Caculate the expect point
    qa = 10**(home_point/400)
    qb =  10**(away_point/400)
    Ea = qa/(qa+qb)
    Eb = qb/(qa+qb)

    # new point of Home team and away team
    Ra_ = round(home_point + weight*(Aa-Ea),2)
    Rb_ = round(away_point + weight*(Ab-Eb),2)

    # Update team point 
    if home in team_dict.keys() and away in team_dict.keys():
        team_dict[home] = Ra_
        team_dict[away] = Rb_
    return [Ra_,Rb_]

In [29]:
final_df[['Home','Away','result']].apply(lambda x: update_ELO_point(home=x['Home'], away=x['Away'], result=x['result']),axis=1)

0        [1485.0, 1515.0]
1        [1515.0, 1485.0]
2        [1470.0, 1500.0]
3        [1530.0, 1500.0]
4        [1515.0, 1485.0]
              ...        
978    [1623.13, 1704.55]
979    [1606.96, 1715.91]
980    [1717.19, 1619.09]
981    [1761.66, 1691.44]
982    [1596.57, 1727.58]
Length: 983, dtype: object

In [30]:
final_df

,Date,Time,Tournament_id,round_id,Home,Away,home_perf_wm,away_perf_wm,rank_diff,Home_streak_score,Away_streak_score,Home_diff_score,Away_diff_score,result,pen_result
0,2019-07-02,21:00:00,5,4,England,USA,0.716496,0.555556,-49.27,1.000000,1.000000,1.990424,3.429089,A,0
1,2019-07-03,21:00:00,5,4,Netherlands,Sweden,0.834743,0.501471,-69.24,1.030842,0.949214,1.362714,0.959822,H,0
2,2019-07-06,17:00:00,5,5,England,Sweden,0.716496,0.501471,-8.95,0.936998,0.719511,1.415979,0.579900,A,0
3,2019-07-07,17:00:00,5,6,USA,Netherlands,0.910757,0.479614,109.56,1.000000,1.154977,2.974879,1.293187,H,0
4,2020-03-05,18:30:00,6,1,Malta,Georgia,0.420000,0.000000,131.97,-0.945487,-1.000000,-3.010254,-5.801186,H,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,2023-08-11,19:30:00,16,3,Japan,Sweden,0.705499,0.501471,-133.03,0.979878,0.395191,2.590036,1.032195,A,0
979,2023-08-12,17:00:00,16,3,Australia,France,0.401938,0.387168,-106.96,0.054874,0.572759,1.436822,1.393902,D,H
980,2023-08-12,20:30:00,16,3,England,Colombia,0.716496,0.368394,338.12,0.307864,0.523335,0.762022,0.778454,H,0
981,2023-08-15,20:00:00,16,4,Spain,Sweden,0.923079,0.501471,-47.43,0.784536,0.875788,1.393845,1.468319,H,0


In [31]:
my_elo_rank = pd.DataFrame(team_dict.items(),columns=['Team','ELO_point']).sort_values(by='ELO_point',ascending=False).reset_index(drop=True)
my_elo_rank['Date'] = pd.to_datetime('today').normalize()
my_elo_rank.head(50)

,Team,ELO_point,Date
0,Spain,1761.66,2023-08-23
1,USA,1744.41,2023-08-23
2,England,1727.58,2023-08-23
3,France,1715.91,2023-08-23
4,Sweden,1691.44,2023-08-23
5,Netherlands,1677.43,2023-08-23
6,Germany,1658.01,2023-08-23
7,Brazil,1644.08,2023-08-23
8,Russia,1630.05,2023-08-23
9,Japan,1623.13,2023-08-23


In [32]:
league_df = database.export_table_to_dataframe(table_name='league_tb')

 Table 'league_tb' is exported to Dataframe successfully.


In [33]:
league_df['weight'] = league_df['Tournament_id']+league_df['round_id']

In [36]:
league_df

,Tournament_id,Tournament,round_id,Round,tour_start,weight
0,1,Algarve Cup,1,Preliminary round,2022,2
1,1,Algarve Cup,2,Classification round,2022,3
2,1,Algarve Cup,3,Final,2022,4
3,2,AFC Women's Asian Cup qualification,1,not_available,2021,3
4,3,AFC Women's Asian Cup,1,Group stage,2022,4
5,3,AFC Women's Asian Cup,2,Play-offs,2022,5
6,3,AFC Women's Asian Cup,3,Quarter-finals,2022,6
7,3,AFC Women's Asian Cup,4,Semi-finals,2022,7
8,3,AFC Women's Asian Cup,5,Final,2022,8
9,4,SheBelieves Cup,1,not_available,2023,5


In [45]:
test = league_df[['Tournament_id','Tournament','tour_start']].sort_values(by='Tournament_id').drop_duplicates().reset_index(drop=True)
test

,Tournament_id,Tournament,tour_start
0,1,Algarve Cup,2022
1,2,AFC Women's Asian Cup qualification,2021
2,3,AFC Women's Asian Cup,2022
3,4,SheBelieves Cup,2023
4,5,2019 FIFA Women's World Cup,2019
5,6,UEFA Women's Euro Qualification,2019
6,7,2021 Friendlies (W),2021
7,8,2022 Friendlies (W),2022
8,9,WCQ — UEFA (W),2021
9,10,Friendlies (W),2023


In [53]:
test['tournament_name'] = test['Tournament'].str.extract(pat=r'(\d*)(.+)',expand=True)[1].str.strip()

In [56]:
test.sort_values(by=['tour_start','tournament_name'])

,Tournament_id,Tournament,tour_start,tournament_name
4,5,2019 FIFA Women's World Cup,2019,FIFA Women's World Cup
5,6,UEFA Women's Euro Qualification,2019,UEFA Women's Euro Qualification
1,2,AFC Women's Asian Cup qualification,2021,AFC Women's Asian Cup qualification
6,7,2021 Friendlies (W),2021,Friendlies (W)
12,13,WCQ — CONCACAF (M),2021,WCQ — CONCACAF (M)
8,9,WCQ — UEFA (W),2021,WCQ — UEFA (W)
2,3,AFC Women's Asian Cup,2022,AFC Women's Asian Cup
10,11,Africa Women Cup of Nations,2022,Africa Women Cup of Nations
0,1,Algarve Cup,2022,Algarve Cup
13,14,CONCACAF W Championship,2022,CONCACAF W Championship
